In [230]:
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath("..")  # Adjust based on your folder structure
sys.path.append(project_root)

In [254]:
import numpy as np
import importlib
import torch
from tactix.utils import *
from tactix.tactixGame import TactixGame
from tactix.tactixLogic import Board
from tactix.tactixMove import Move

importlib.reload(sys.modules['tactix.tactixGame'])
importlib.reload(sys.modules['tactix.utils'])

<module 'tactix.utils' from '/Users/alibal/Desktop/tactix-game/tactix/utils.py'>

In [232]:
from tactix.utils import encode_action, decode_action

class TactixEnvironment:
    def __init__(self, board_size = 5):
        """Initialize the environment with a game instance."""
        self.game = TactixGame(height=board_size, width=board_size)  # Initialize the game logic
        self.state = None  # Current game state
        self.done = False  # Flag to indicate if the game is over
        self.starting_player = 1

    def reset(self):
        """Reset the environment to the initial state."""
        self.starting_player = -1 if self.starting_player == 1 else 1
        self.game = TactixGame(current_player=self.starting_player)  # Create a new instance of TactixGame
        self.state = self.game.getPieces()  # Initialize the board state
        self.done = False  # Reset the game-over flag
        valid_moves_mask = self._generate_valid_moves_mask()
        return self._get_observation(), valid_moves_mask

    def step(self, action):
        """Execute the action in the environment."""
        move = decode_action(action, self.game.height)  # Decode action index
        self.game.makeMove(move)  # Execute the move
        game_ended = self.game.getGameEnded()

        if game_ended and game_ended.is_ended:
            reward = -1
            self.done = True
        else:
            opponent_move = self.game.get_random_move()
            self.game.makeMove(opponent_move)
            game_ended = self.game.getGameEnded()
            if game_ended and game_ended.is_ended:
                reward = 1
                self.done = True
            else:
                reward = 0

        self.state = self.game.getPieces()
        valid_moves_mask = self._generate_valid_moves_mask()
        return self._get_observation(), reward, self.done, valid_moves_mask

    def _generate_valid_moves_mask(self):
        valid_moves = self.game.getValidMoves()
        valid_moves_mask = torch.zeros(self.game.height ** 3)
        for move in valid_moves:
            action_index = encode_action(move, self.game.height)
            valid_moves_mask[action_index] = 1
        return valid_moves_mask

    def _get_observation(self):
        """
        Convert the current board state into a PyTorch tensor.

        Returns:
            torch.Tensor: The current state as a tensor.
        """
        return torch.from_numpy(np.array(self.state, dtype=np.float32))

    def render(self):
        """Display the current board and game status."""
        self.game.display()

In [234]:
def eski_decode_action(action_index, n):
    """
    Invert the encoding scheme:
      - If action_index < n * sum_{k=1}^n(k) = 75 for n=5, it is a horizontal move.
      - Otherwise it is a vertical move (shifted by 75).

    Returns:
      Move(row, col, piece_count, ver)
    """
    # Some helpers
    S1 = sum(range(1, n+1))  # For n=5, S1 = 15
    S2 = sum(range(1, n))    # For n=5, S2 = 10

    # Decide horizontal vs vertical
    if action_index < n * S1:
        # -----------------
        # Horizontal moves
        # -----------------
        # Each row has 15 possible horizontal “sub-indices”.
        #   row part:   row = action_index // 15
        #   partial:    partial_index = action_index % 15
        ver = False

        row = action_index // S1
        partial_index = action_index % S1


        start = 0
        count = 1
        k = 0
        while k < partial_index + 1:
            while start + count < n+1:
                if k == partial_index:
                    return Move (row = row, col = start, piece_count = count, ver = ver)
                count += 1
                k += 1
            start += 1
            count = 1
            
        return Move(row=row, col=col, piece_count=piece_count, ver=ver)

    else:
        # ---------------
        # Vertical moves
        # ---------------
        # They start at index = n*S1 = 75 for n=5.
        # So subtract 75 to get a 0-based index for vertical moves.
        #   i = action_index - 75
        #   col part:  col = i // 10   (since S2=10 for n=5)
        #   partial:   p  = i % 10
        # Then we decode (row, piece_count) out of that partial.
        ver = True

        offset = action_index - n * S1  # e.g. offset in [0..49] for n=5
        col = offset // S2
        partial_index = offset % S2

        start = 0
        count = 2 # one piece removals are calculated on horizontal moves
        k = 0
        while k < partial_index + 1:
            while start + count < n+1:
                if k == partial_index:
                    return Move (row = start, col = col, piece_count = count, ver = ver)
                count += 1
                k += 1
            start += 1
            count = 2   

    return Move(row=row, col=col, piece_count=piece_count, ver=ver)